In [6]:
from os import environ
from pathlib import Path
from json import loads, dumps
from random import choice
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()
from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: True, maxTries = 10, temperature = 1):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query, temperature)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [7]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [11]:
class QAGraph:
    def __init__(self):
        self.idCounter = 0
        self.nodes = []
        self.edges = []
    def question(self, question):
        self.idCounter += 1
        node =  QuestionNode(question, self, self.idCounter)
        self.nodes.append(node)
        return node
    def statement(self, statement):
        self.idCounter += 1
        node = StatementNode(statement, self, self.idCounter)
        self.nodes.append(node)
        return node
    def topic(self, topic):
        self.idCounter += 1
        node = TopicNode(topic, self, self.idCounter)
        self.nodes.append(node)
        return node
    def generateGraph(self, path):
        nodes = []
        edges = []
        for node in self.nodes:
            node.generateVisNode(nodes, edges)
        for edge in self.edges:
            edge.generateVisEdge(nodes, edges)
        jsonData = {"nodes": nodes, "edges": edges}
        safeVisNetworkJSONToHTMLFile(jsonData, path)
    def hasSubStatement(self, node1, node2):
        self.idCounter += 1
        edge = HasSubStatementEdge(node1, node2, self, self.idCounter)
        self.edges.append(edge)
        return edge
    def isContradictionTo(self, node1, node2):
        self.idCounter += 1
        edge = IsContradictionToEdge(node1, node2, self, self.idCounter)
        self.edges.append(edge)
        return edge
    def isTopicOf(self, node1, node2):
        self.idCounter += 1
        edge = HasTopicEdge(node1, node2, self, self.idCounter)
        self.edges.append(edge)
        return edge
    def answers(self, node1, node2):
        self.idCounter += 1
        edge = AnswersEdge(node1, node2, self, self.idCounter)
        self.edges.append(edge)
        return edge
from math import sqrt, ceil
def breakTextIntoNewLineSeparatedChunks(text, maxChars = None):
    if maxChars is None:
        maxChars = ceil(sqrt(len(text))) * 3
    result = ""
    while len(text) > 0:
        lastSpaceIndex = 0
        i = 0
        while i < len(text) and (i < maxChars or lastSpaceIndex == 0):
            if text[i] == " ":
                lastSpaceIndex = i
            i += 1
        i = lastSpaceIndex if i != len(text) else i
        result += text[:i] + "\n"
        text = text[i + 1:] if i < len(text) else ""
    return result
    
class QuestionNode:
    def __init__(self, question, graph, id):
        self.question = question
        self.graph = graph
        self.id = id
    def generateVisNode(self, nodes, edges):
        nodes.append({"id": self.id, "label": "Question:\n" + breakTextIntoNewLineSeparatedChunks(self.question), "shape": "box", "color": "rgb(120, 190, 255)"})
class StatementNode:
    def __init__(self, statement, graph, id):
        self.statement = statement
        self.graph = graph
        self.id = id
    def generateVisNode(self, nodes, edges):
        nodes.append({"id": self.id, "label": "Statement:\n" + breakTextIntoNewLineSeparatedChunks(self.statement), "shape": "box", "color": "rgb(255, 190, 120)"})
    def answeres(self, node):
        self.graph.idCounter += 1
        edge = AnswersEdge(self, node, self.graph, self.graph.idCounter)
        self.graph.edges.append(edge)
        return edge
    def hasSubStatement(self, node):
        self.graph.idCounter += 1
        edge = HasSubStatementEdge(self, node, self.graph, self.graph.idCounter)
        self.graph.edges.append(edge)
        return edge
    def isContradictionTo(self, node):
        self.graph.idCounter += 1
        edge = IsContradictionToEdge(self, node, self.graph, self.graph.idCounter)
        self.graph.edges.append(edge)
        return edge
class TopicNode:
    def __init__(self, topic, graph, id):
        self.topic = topic
        self.graph = graph
        self.id = id
    def generateVisNode(self, nodes, edges):
        nodes.append({"id": self.id, "label": "Topic:\n" + breakTextIntoNewLineSeparatedChunks(self.topic), "shape": "box", "color": "rgb(120, 255, 120)"})
    def isTopicOf(self, node):
        self.graph.idCounter += 1
        edge = HasTopicEdge(node, self, self.graph, self.graph.idCounter)
        self.graph.edges.append(edge)
        return edge
class AnswersEdge:
    def __init__(self, node1, node2, graph, id):
        self.node1 = node1
        self.node2 = node2
        self.graph = graph
        self.id = id
    def generateVisEdge(self, nodes, edges):
        nodes.append({"id": self.id, "label": "answers", "shape": "box", "color": "rgb(255, 210, 80)"})
        edges.append({"from": self.node1.id, "to": self.id, "color": "rgb(255, 210, 80)"})
        edges.append({"from": self.id, "to": self.node2.id, "arrows": "to", "color": "rgb(255, 210, 80)"})
class HasTopicEdge:
    def __init__(self, node1, node2, graph, id):
        self.node1 = node1
        self.node2 = node2
        self.graph = graph
        self.id = id
    def generateVisEdge(self, nodes, edges):
        nodes.append({"id": self.id, "label": "has topic", "shape": "box", "color": "rgb(200, 235, 90)"})
        edges.append({"from": self.node1.id, "to": self.id, "color": "rgb(200, 235, 90)"})
        edges.append({"from": self.id, "to": self.node2.id, "arrows": "to", "color": "rgb(200, 235, 90)"})
class HasSubStatementEdge:
    def __init__(self, node1, node2, graph, id):
        self.node1 = node1
        self.node2 = node2
        self.graph = graph
        self.id = id
    def generateVisEdge(self, nodes, edges):
        nodes.append({"id": self.id, "label": "has substatement", "shape": "box", "color": "rgb(255, 190, 220)"})
        edges.append({"from": self.node1.id, "to": self.id, "color": "rgb(255, 190, 220)"})
        edges.append({"from": self.id, "to": self.node2.id, "arrows": "to", "color": "rgb(255, 190, 220)"})
class IsContradictionToEdge:
    def __init__(self, node1, node2, graph, id):
        self.node1 = node1
        self.node2 = node2
        self.graph = graph
        self.id = id
    def generateVisEdge(self, nodes, edges):
        nodes.append({"id": self.id, "label": "is contradiction to", "shape": "box", "color": "rgb(255, 120, 120)"})
        edges.append({"from": self.node1.id, "to": self.id, "arrows": "from", "color": "rgb(255, 120, 120)"})
        edges.append({"from": self.id, "to": self.node2.id, "arrows": "to", "color": "rgb(255, 120, 120)"})

In [27]:
print(breakTextIntoNewLineSeparatedChunks("This is a test of the emergency broadcast system. This is only a test. If this were a real emergency, you would be instructed to do something else."))

This is a test of the emergency
broadcast system. This is only a test.
If this were a real emergency, you
would be instructed to do something
else.



In [76]:
qag = QAGraph()
l = qag.topic("Light")
sr = qag.topic("Special Relativity")
cm = qag.topic("Classical Mechanics")
wisl = qag.question("What is the speed of light?")
l.isTopicOf(wisl)
tsoli = qag.statement("The speed of light is 299,792,458 meters per second in a vacuum.")
tsoli.answeres(wisl)
l.isTopicOf(tsoli)
lwctttv = qag.statement("Light waves can travel through the vacuum.")
tsoli.hasSubStatement(lwctttv)
l.isTopicOf(lwctttv)
tsoliac = qag.statement("The speed of light is a natural constant that is independent of the inertial frame of reference.")
tsoliac.answeres(wisl)
l.isTopicOf(tsoliac)
sr.isTopicOf(tsoliac)
wqii = qag.question("What quantity is independent of the inertial frame of reference?")
tsoliac.answeres(wqii)
tiiotifr = qag.statement("Time is independent of the inertial frame of reference.")
tiiotifr.answeres(wqii)
cm.isTopicOf(tiiotifr)
tmtbted = qag.statement("The measured time between two events depends on the observer's inertial frame of reference.")
tiiotifr.isContradictionTo(tmtbted)
sr.isTopicOf(tmtbted)
qag.generateGraph("output.html")

In [2]:
def generateGraphForTopic(topic):
    query = '''
You have the task of creating a semantic network of questions, statements, and topics related to the topic "''' + topic + '''".
The created network should have the following json structure:
{
    "questions": { ... },
    "statements": { ... },
    "topics": { ... },
    "edges": [ ... ]
}
The "questions", "statements", and "topics" objects are dictionaries with the keys being the ids of the nodes and the values being the text of the nodes.
The "edges" list contains triples of the form [node1_id, edge_type, node2_id] where "edge_type" is one of "answers", "has_topic", "has_substatement", or "is_contradiction_to".
An example of a valid json structure that represents a semantic network about the topic "Special Relativity" is:
{
    "questions": {
        "1": "What is the speed of light?",
        "2": "What quantity is independent of the inertial frame of reference?"
    },
    "statements": {
        "3": "The speed of light is 299,792,458 meters per second in a vacuum.",
        "4": "Light waves can travel through the vacuum.",
        "5": "The speed of light is a natural constant that is independent of the inertial frame of reference.",
        "6": "Time is independent of the inertial frame of reference.",
        "7": "The measured time between two events depends on the observer's inertial frame of reference."
    },
    "topics": {
        "8": "Light",
        "9": "Special Relativity",
        "10": "Classical Mechanics"
    },
    "edges": [
        ["3", "answers", "1"],
        ["3", "has_topic", "8"],
        ["3", "has_substatement", "4"],
        ["4", "has_topic", "8"],
        ["5", "answers", "1"],
        ["5", "answers", "2"],
        ["5", "has_topic", "8"],
        ["5", "has_topic", "9"],
        ["6", "answers", "2"],
        ["6", "has_topic", "10"],
        ["7", "answers", "2"],
        ["7", "is_contradiction_to", "6"],
        ["7", "has_topic", "9"]
    ]
}
Return nothing but the generated json structure without explanation.
'''
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, dict)
        assert "questions" in result
        assert "statements" in result
        assert "topics" in result
        assert "edges" in result
        allKeys = set()
        for key in result["questions"]:
            assert not key in allKeys
            allKeys.add(key)
            assert isinstance(key, str)
            assert isinstance(result["questions"][key], str)
        for key in result["statements"]:
            assert not key in allKeys
            allKeys.add(key)
            assert isinstance(key, str)
            assert isinstance(result["statements"][key], str)
        for key in result["topics"]:
            assert not key in allKeys
            allKeys.add(key)
            assert isinstance(key, str)
            assert isinstance(result["topics"][key], str)
        for edge in result["edges"]:
            assert isinstance(edge, list)
            assert len(edge) == 3
            assert edge[0] in allKeys
            assert edge[1] in ["answers", "has_topic", "has_substatement", "is_contradiction_to"]
            assert edge[2] in allKeys
        return result
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    return answer

In [12]:
def createQAGraphForJSONGraph(jsonGraph):
    qag = QAGraph()
    nodeById = {}
    for key, text in jsonGraph["questions"].items():
        node = qag.question(text)
        nodeById[key] = node
    for key, text in jsonGraph["statements"].items():
        node = qag.statement(text)
        nodeById[key] = node
    for key, text in jsonGraph["topics"].items():
        node = qag.topic(text)
        nodeById[key] = node
    for edge in jsonGraph["edges"]:
        if edge[1] == "answers":
            qag.answers(nodeById[edge[0]], nodeById[edge[2]])
        elif edge[1] == "has_topic":
            qag.isTopicOf(nodeById[edge[0]], nodeById[edge[2]])
        elif edge[1] == "has_substatement":
            qag.hasSubStatement(nodeById[edge[0]], nodeById[edge[2]])
        elif edge[1] == "is_contradiction_to":
            qag.isContradictionTo(nodeById[edge[0]], nodeById[edge[2]])
    return qag

In [4]:
def createGraphHtmlAboutTopic(topic, outputPath):
    jsonGraph = generateGraphForTopic(topic)
    qag = createQAGraphForJSONGraph(jsonGraph)
    qag.generateGraph(outputPath)

In [32]:
createGraphHtmlAboutTopic("Bell's theorem", "output.html")